# Creating Orders (CFD Trading)

Please run the following code only with your Paper Trading Account!!

Check the Regular Trading Hours!!!

In [21]:
import pandas as pd
from ib_async import *
util.startLoop()

In [22]:
ib = IB()

In [23]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

Error 10349, reqId 22: Order TIF was set to DAY based on order preset.
Canceled order: Trade(contract=CFD(conId=143916318, symbol='EUR', exchange='SMART', currency='USD', localSymbol='EUR.USD', tradingClass='EUR.USD'), order=MarketOrder(orderId=22, clientId=1, action='BUY', totalQuantity=10000), orderStatus=OrderStatus(orderId=22, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2025, 11, 24, 20, 17, 41, 550270, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2025, 11, 24, 20, 17, 41, 556700, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 10349, reqId 22: Order TIF was set to DAY based on order preset.', errorCode=10349)], advancedError='')
Error 201, reqId 22: Order rejected - reason:No Trading Permission, Regulatory Restriction. Please contact cust

In [24]:
contract = CFD("EUR", currency = "USD")
contract

CFD(symbol='EUR', currency='USD')

In [25]:
ib.qualifyContracts(contract)

[CFD(conId=143916318, symbol='EUR', exchange='SMART', currency='USD', localSymbol='EUR.USD', tradingClass='EUR.USD')]

Going Long

In [26]:
order = MarketOrder(action = "BUY", totalQuantity = 10000)
order

MarketOrder(action='BUY', totalQuantity=10000)

In [27]:
trade = ib.placeOrder(contract, order)
trade

Trade(contract=CFD(conId=143916318, symbol='EUR', exchange='SMART', currency='USD', localSymbol='EUR.USD', tradingClass='EUR.USD'), order=MarketOrder(orderId=22, clientId=1, action='BUY', totalQuantity=10000), orderStatus=OrderStatus(orderId=22, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2025, 11, 24, 20, 17, 41, 550270, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')

In [28]:
order = MarketOrder(action = "SELL", totalQuantity = 10000)

## CFD Trade Information

In [29]:
ib.fills()

[]

In [30]:
util.df([fs.execution for fs in ib.fills()])  # execution data

In [31]:
fills = ib.fills()
if fills:
    fill_df = util.df([fs.execution for fs in fills])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    fill_df
else:
    print("No fills available yet. Execute a trade first.")
    fill_df = None

No fills available yet. Execute a trade first.


In [32]:
if fill_df is not None:
    fill_df["TradeValue"] = fill_df.side.apply(lambda x: 1 if x == "SLD" else -1) * fill_df.cumQty * fill_df.avgPrice
    fill_df
else:
    print("No fill data to calculate trade values.")

No fill data to calculate trade values.


In [33]:
if fill_df is not None:
    fill_df.TradeValue.sum()
else:
    print("No trade value to sum.")

No trade value to sum.


In [34]:
util.df([fs.commissionReport for fs in ib.fills()])  # commission and PNL data

In [35]:
fills = ib.fills()
if fills:
    profit_df = util.df([fs.commissionReport for fs in fills])[["execId", "currency", "commission", "realizedPNL"]].set_index("execId")
    profit_df
else:
    print("No commission data available yet.")
    profit_df = None

No commission data available yet.


In [36]:
if profit_df is not None:
    profit_df.realizedPNL.sum()
else:
    print("No PNL data to sum.")

No PNL data to sum.


Total Commissions (in USD) + Sum of Trade Values == Total PNL

In [39]:
if profit_df is not None and fill_df is not None:
    total_pnl = - profit_df.commission.sum() * fill_df.avgPrice.mean() + fill_df.TradeValue.sum()
    print(f"Total PNL: {total_pnl}")
else:
    print("Cannot calculate PNL - no trade data available yet.")

Cannot calculate PNL - no trade data available yet.


In [40]:
ib.disconnect()

'Disconnecting from 127.0.0.1:7497, 369 B sent in 10 messages, 32.1 kB received in 611 messages, session time 208 s.'